Script to generate a database of all NHL players season-by-season data from 1980 to today. Also includes code to scrape NHL average team data for each relevant year.

In [2]:
from bs4 import BeautifulSoup, Comment
import pandas as pd
import requests
import string
from time import sleep

In [7]:
# Get player season data
df = pd.DataFrame()
site = 'https://www.hockey-reference.com'
prev_player=''
for letter in string.ascii_lowercase:
    count = 0
    url = requests.get(site+'/players/'+letter+'/').text
    soup = BeautifulSoup(url,'lxml')
    player_list = soup.find_all('p', class_='nhl')

    for count, player in enumerate(player_list):
        sleep(0.1)
        # Skip goalies and anybody from before 1980
        if int(player.text.split('(')[-1][0:4])<1980 or player.text[-2] == 'G':
            continue
        # If players have the same name, differentiate them
        if player.a.text == prev_player:
            if same_name_flag == True:
                same_name_count += 1
            same_name_flag = True
        else:
            same_name_flag = False
            same_name_count = 0
        player_name = player.a.text+same_name_flag*str(same_name_count)
        prev_player = player.a.text
        
        print(f'\r{count:03d}/{len(player_list):03d} of the letter {letter.upper()} completed, currently on: {player_name}                           ',end='')
        
        # Access player webpage and get season data table
        player_url = site+player.a['href']
        player_url = requests.get(player_url).text
        try:
            df_list = pd.read_html(player_url, skiprows = 1, header = 0)
        except:
            continue
        tmp_df = df_list[0]
        # Only grab NHL seasons
        tmp_df = tmp_df[tmp_df.Lg == 'NHL']
        tmp_df.drop(columns = ['Lg','Awards'], inplace=True)
        if tmp_df['Age'].isnull().any():
            tmp_df.drop(tmp_df.loc[tmp_df['Age'].isnull()].index, inplace=True)
        tmp_df['Player'] = player_name
        if df.empty:
            df = tmp_df
        else:
            df = df.append(tmp_df, ignore_index=True)
print('\rCompleted!                                                              ',end='')

011/225 of the letter A completed, currently on: Thommy Abrahamsson                           

C:\Users\aronf\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\aronf\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Completed! the letter Z completed, currently on: Andrei Zyuzin                                           


In [14]:
df.to_csv('NHL_player_season_log_all.csv',index=False)

In [170]:
# Get league average season data
site = 'https://www.hockey-reference.com'
stat_list = ['games','total_goals_per_game','goals_pp','chances_pp','shots','shot_pct']
stat_dict = {
    'year': [],
    'games': [],
    'total_goals_per_game': [],
    'goals_pp': [],
    'chances_pp': [],
    'shots': [],
    'shot_pct': []
}

for year in range(1977,2021):
    if year == 2005:
        continue # Stupid lockout
    sleep(0.1)
    print(f'\r{year}',end='')
    year_url = site+'/leagues/NHL_'+str(year)+'.html'
    url = requests.get(year_url).text
    
    soup = BeautifulSoup(url,'lxml')
    stat_dict['year'].append(f'{year-1}-{str(year)[-2:]}') 
    # Tables are in HTML comments, must use workaround to access
    for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find("table")
            table2=table.find('tfoot')
            for stat in stat_list:
                stat_dict[stat].append(table2.find('td',attrs={'data-stat':stat}).text)
            break
season_df = pd.DataFrame(stat_dict)

2020

In [171]:
season_df.to_csv('League_Average_Data.csv',index=False)